In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# American Option Pricing with the Binomial Model

This notebook contains my solution to **Module 5** of the Coursera course  
[*Introduction to Financial Engineering and Risk Management*](https://www.coursera.org/learn/financial-engineering-intro/home/info)

The module is dedicated to the **Binomial Model for option pricing**, and this notebook specifically focuses on **American options**, emphasizing the treatment of **early exercise** using a recombining binomial tree. The goal is to provide a clear and educational implementation that closely follows the theoretical framework presented in the course.

In [2]:
N = 15        # number of time steps in the binomial tree
T = 0.25      # time to maturity (in years)
S0 = 100      # initial underlying asset price
r = 0.02      # continuously compounded risk-free interest rate
sigm = 0.3    # volatility of the underlying asset
c = 0.01      # continuous dividend yield
K = 110.0     # option strike price

In [3]:
def crr_ud(sigm, T, N):
    """
    Compute the up and down factors in the Cox–Ross–Rubinstein (CRR) binomial model.

    Parameters
    ----------
    sigm : float
        Volatility of the underlying asset.
    T : float
        Time to maturity (in years).
    N : int
        Number of time steps in the binomial tree.

    Returns
    -------
    u : float
        Upward movement factor per time step.
    d : float
        Downward movement factor per time step (reciprocal of u).
    dt : float
        Length of each time step.
    """
    dt = T / N
    u = np.exp(sigm * np.sqrt(dt))   # up factor
    d = 1.0 / u                     # down factor
    return u, d, dt

In [4]:
u, d, dt = crr_ud(sigm, T, N)   # up and down factors

In [5]:
u,d, dt

(np.float64(1.0394896104013376),
 np.float64(0.9620105771080376),
 0.016666666666666666)

In [6]:
def stock_tree(S0, u, d, N, c):
    """
    Construct the recombining binomial tree of underlying asset prices.

    Parameters
    ----------
    S0 : float
        Initial price of the underlying asset.
    u : float
        Upward movement factor per time step.
    d : float
        Downward movement factor per time step.
    N : int
        Number of time steps in the binomial tree.
    c : float
        Continuous dividend yield (included for model completeness; not explicitly
        applied in the stock price evolution).

    Notes
    -----
    At time step i, the asset price after j downward moves is:
        S(i, j) = S0 * u^(i - j) * d^j

    The tree is stored as an (N+1) x (N+1) matrix where:
    - Rows correspond to time steps i = 0, ..., N
    - Columns correspond to the number of down moves j = 0, ..., i
    - Entries with j > i are set to NaN

    Returns
    -------
    tree : numpy.ndarray
        Matrix containing the binomial stock price tree.
    """
    tree = np.full((N + 1, N + 1), np.nan, dtype=float)

    for i in range(N + 1):
        j = np.arange(i + 1)                       # number of down moves (0 to i)
        tree[i, j] = S0 * (u ** (i - j)) * (d ** j)

    return tree


In [7]:
S = stock_tree(S0, u, d, N, c)

In [8]:
# printing stock price tree

for i in range(N + 1):
    vals = [f"{S[i, j]:8.4f}" for j in range(i + 1)]
    print(f"t={i:2d}: ", " ".join(vals))

t= 0:  100.0000
t= 1:  103.9490  96.2011
t= 2:  108.0539 100.0000  92.5464
t= 3:  112.3209 103.9490  96.2011  89.0306
t= 4:  116.7564 108.0539 100.0000  92.5464  85.6484
t= 5:  121.3670 112.3209 103.9490  96.2011  89.0306  82.3947
t= 6:  126.1598 116.7564 108.0539 100.0000  92.5464  85.6484  79.2646
t= 7:  131.1418 121.3670 112.3209 103.9490  96.2011  89.0306  82.3947  76.2534
t= 8:  136.3205 126.1598 116.7564 108.0539 100.0000  92.5464  85.6484  79.2646  73.3565
t= 9:  141.7038 131.1418 121.3670 112.3209 103.9490  96.2011  89.0306  82.3947  76.2534  70.5698
t=10:  147.2996 136.3205 126.1598 116.7564 108.0539 100.0000  92.5464  85.6484  79.2646  73.3565  67.8889
t=11:  153.1164 141.7038 131.1418 121.3670 112.3209 103.9490  96.2011  89.0306  82.3947  76.2534  70.5698  65.3098
t=12:  159.1629 147.2996 136.3205 126.1598 116.7564 108.0539 100.0000  92.5464  85.6484  79.2646  73.3565  67.8889  62.8287
t=13:  165.4482 153.1164 141.7038 131.1418 121.3670 112.3209 103.9490  96.2011  89.0306  8

In [9]:
def get_p(r, c, T, N, u, d):
    """
    Compute the risk-neutral probability of an upward move in the binomial model.

    Parameters
    ----------
    r : float
        Continuously compounded risk-free interest rate.
    c : float
        Continuous dividend yield of the underlying asset.
    T : float
        Time to maturity (in years).
    N : int
        Number of time steps in the binomial tree.
    u : float
        Upward movement factor per time step.
    d : float
        Downward movement factor per time step.

    Notes
    -----
    Under the risk-neutral measure with continuous dividends, the probability p
    satisfies:
        E[S_{t+dt} / S_t] = exp((r - c) * dt)

    The resulting CRR risk-neutral probability is:
        p = (exp((r - c) * dt) - d) / (u - d)

    Returns
    -------
    p : float
        Risk-neutral probability of an up move.
    """
    dt = T / N
    return (np.exp((r - c) * dt) - d) / (u - d)

In [10]:
p = get_p(r, c, T, N, u, d)

In [11]:
p

np.float64(0.4924700506245105)

In [12]:
def american_call_option(r, c, T, N, u, d, S, K):
    """
    Price an American call option using a recombining binomial tree.

    Parameters
    ----------
    r : float
        Continuously compounded risk-free interest rate.
    c : float
        Continuous dividend yield of the underlying asset.
    T : float
        Time to maturity (in years).
    N : int
        Number of time steps in the binomial tree.
    u : float
        Upward movement factor per time step.
    d : float
        Downward movement factor per time step.
    S : numpy.ndarray
        Binomial tree of underlying asset prices with shape (N+1, N+1).
    K : float
        Option strike price.

    Notes
    -----
    The option value is computed via backward induction under the risk-neutral
    measure. At each node, the American option value is the maximum between:
    - the continuation value (expected discounted future payoff), and
    - the immediate exercise value.

    The continuation value at time step i is given by:
        disc * [ p * V(i+1, j) + (1 - p) * V(i+1, j+1) ]

    where p is the risk-neutral probability of an up move.

    Returns
    -------
    V : numpy.ndarray
        Option values at time 0 (V[0] is the option price).
    """
    # risk-neutral probability
    p = get_p(r, c, T, N, u, d)

    # discount factor per time step
    disc = np.exp(-r * dt)

    # number of time steps inferred from the stock price tree
    N = S.shape[0] - 1

    # terminal payoff at maturity (valid nodes j = 0..N)
    V = np.maximum(S[N, :N+1] - K, 0.0)

    # backward induction through the tree
    for i in range(N - 1, -1, -1):
        # continuation value for nodes j = 0..i
        cont = disc * (p * V[:i+1] + (1 - p) * V[1:i+2])

        # immediate exercise value at time step i
        exer = np.maximum(S[i, :i+1] - K, 0.0)

        # American option value at time step i
        V = np.maximum(cont, exer)

        # Diagnostic output to identify early exercise
        print("\nTime step i =", i)
        print("Immediate exercise value:", exer)
        print("Continuation value:", cont)

    return V

In [13]:
cv = american_call_option(r, c, T, N, u, d, S, K)   # call option value


Time step i = 14
Immediate exercise value: [61.98166193 49.16289705 37.29958714 26.32051674 16.15977848  6.75637744
  0.          0.          0.          0.          0.          0.
  0.          0.          0.        ]
Continuation value: [61.98966127 49.17303267 37.31169981 26.3344591  16.17541416  6.77358022
  1.14257807  0.          0.          0.          0.          0.
  0.          0.          0.        ]

Time step i = 13
Immediate exercise value: [55.44817785 43.11639045 31.70376083 21.14177898 11.3670413   2.32087004
  0.          0.          0.          0.          0.          0.
  0.          0.        ]
Continuation value: [55.46634654 43.13866905 31.72984301 21.17138123 11.39990126  3.91437298
  0.56249795  0.          0.          0.          0.          0.
  0.          0.        ]

Time step i = 12
Immediate exercise value: [49.16289705 37.29958714 26.32051674 16.15977848  6.75637744  0.
  0.          0.          0.          0.          0.          0.
  0.        ]
Cont

In [14]:
round(cv[0],2)

np.float64(2.6)

In [15]:
def american_put_option(r, c, T, N, u, d, S, K):
    """
    Price an American put option with a recombining binomial tree (early exercise allowed).

    Key idea (why early exercise matters for puts)
    ----------------------------------------------
    Unlike a non-dividend-paying call, an American *put* can be optimal to exercise early.
    If the put is sufficiently in-the-money, exercising converts the option into cash (K)
    sooner, which can be invested at the risk-free rate r. This benefit can outweigh the
    remaining time value of holding the option.

    This function also supports answering:
      1) "Does early exercise ever make sense?"  -> Yes, if at some node exercise value > continuation value.
      2) "What is the earliest period to do that?" -> The smallest time index i where exercise is optimal
         at at least one node (i.e., where exer > cont).

    Parameters
    ----------
    r : float
        Continuously compounded risk-free interest rate.
    c : float
        Continuous dividend yield (used in risk-neutral probability; put early exercise logic is still driven
        by comparing continuation vs exercise).
    T : float
        Time to maturity (years).
    N : int
        Number of time steps.
    u, d : float
        Up/down movement factors per step.
    S : numpy.ndarray
        Stock price tree of shape (N+1, N+1), where S[i, j] is price at time i with j down moves.
    K : float
        Strike price.

    Returns
    -------
    V : numpy.ndarray
        Option values at time 0 (V[0] is the option price).

    (Optional diagnostic you can add)
    ---------------------------------
    To find the earliest exercise period, track the first i (going forward in time)
    such that exer > cont at any node. In this backward loop, that corresponds to
    the *smallest i* for which the condition holds; you can store it in a variable
    like earliest_exercise_i.
    """
    # Risk-neutral probability of an up move (CRR with continuous dividend yield)
    p = get_p(r, c, T, N, u, d)

    # One-step length and discount factor per step
    dt = T / N
    disc = np.exp(-r * dt)

    # Ensure N matches the stock tree depth
    N = S.shape[0] - 1

    # Terminal payoff at maturity: put payoff = max(K - S_T, 0)
    V = np.maximum(K - S[N, :N+1], 0.0)

    # (Optional) track earliest time step where early exercise is optimal at any node
    # earliest_exercise_i = None

    # Backward induction: step from time N-1 down to 0
    for i in range(N - 1, -1, -1):

        # Continuation value: discounted expected value under the risk-neutral measure
        # cont(j) = e^{-r dt} [ p * V_next(j) + (1-p) * V_next(j+1) ]
        cont = disc * (p * V[:i+1] + (1 - p) * V[1:i+2])

        # Immediate exercise value at time i: put intrinsic value = max(K - S(i,j), 0)
        exer = np.maximum(K - S[i, :i+1], 0.0)

        # American put value at each node: choose the better of holding vs exercising
        V = np.maximum(cont, exer)

        # Diagnostic output to identify early exercise
        print("\nTime step i =", i)
        print("Immediate exercise value:", exer)
        print("Continuation value:", cont)

    return V

In [16]:
pv = american_put_option(r, c, T, N, u, d, S, K)


Time step i = 14
Immediate exercise value: [ 0.          0.          0.          0.          0.          0.
  1.94613499 10.         17.45356495 24.3515736  30.73543469 36.64347055
 42.11114712 47.17128687 51.85426581]
Continuation value: [ 0.          0.          0.          0.          0.          0.
  3.07005997  9.98000472 17.43232752 24.32918659 30.7119838  36.61903507
 42.08580043 47.14509689 51.8272954 ]

Time step i = 13
Immediate exercise value: [ 0.          0.          0.          0.          0.          0.
  6.05103896 13.79894229 20.96935061 27.60530789 33.74664979 39.43024277
 44.69020547 49.55811342]
Continuation value: [ 0.          0.          0.          0.          0.          1.55762809
  6.58501671 13.77831391 20.94752726 27.58237864 33.72269706 39.40534286
 44.66442897 49.53152567]

Time step i = 12
Immediate exercise value: [ 0.          0.          0.          0.          0.          1.94613499
 10.         17.45356495 24.3515736  30.73543469 36.64347055 42.111

In [17]:
round(pv[0],2)

np.float64(12.36)

#### let's see if they satisfy put call parity

In [18]:
pv + S0 *np.exp(-c * T) == cv + K *np.exp(-r * T)

array([False])

### Task: American Call on a Futures Contract (Binomial Model)

Compute the **fair value** of an **American call option** with **strike** \(K = 110\) and **option maturity** \(n = 10\) time steps (periods).  
The option is written on a **futures contract** that **expires at period 15**, i.e., the underlying futures has a longer maturity than the option itself (\(15 > 10\)).

The futures contract is assumed to be written on the **same underlying security** and uses the **same market parameters** as in the previous questions.

In [19]:
def american_call_on_futures_from_stock_tree(
    S, K, u, d, r, c, T, N, opt_periods=10, fut_periods=15
):
    """
    Price an American call option written on a futures contract using a binomial model.

    The underlying futures contract is derived from a binomial stock price tree and
    expires later than the option itself. Early exercise of the American call is
    explicitly allowed and tested at every node.

    Financial setting
    -----------------
    - The option expires after `opt_periods` time steps.
    - The underlying futures contract expires after `fut_periods` time steps, with
      fut_periods > opt_periods.
    - The futures price at each node is computed from the spot price using
      cost-of-carry:
          F(t) = S(t) * exp((r - c) * (T_fut - t))

    where r is the risk-free rate and c is the continuous dividend yield.

    Parameters
    ----------
    S : numpy.ndarray
        Binomial tree of underlying spot prices with shape (N+1, N+1).
    K : float
        Strike price of the American call option.
    u, d : float
        Up and down factors per time step.
    r : float
        Continuously compounded risk-free interest rate.
    c : float
        Continuous dividend yield of the underlying asset.
    T : float
        Total time horizon corresponding to N periods.
    N : int
        Total number of periods in the stock price tree.
    opt_periods : int, optional
        Number of periods until option maturity (default is 10).
    fut_periods : int, optional
        Number of periods until futures maturity (default is 15).

    Notes
    -----
    Early exercise may be optimal for an American call on a futures contract, since
    holding the futures itself does not require an upfront payment. This function
    determines endogenously whether early exercise is optimal by comparing, at each
    node:
        - the continuation value, and
        - the immediate exercise value.

    The earliest time step i at which:
        immediate exercise value > continuation value
    indicates the earliest period where early exercise is optimal.

    Returns
    -------
    V : numpy.ndarray
        Option values at time 0 (V[0] is the fair value of the American call).
    """

    # length of one time step
    dt = T / N

    # risk-neutral probability of an up move
    p = get_p(r, c, T, N, u, d)

    # one-step discount factor
    disc = np.exp(-r * dt)

    # Build the futures price tree up to option maturity
    # F[i, j] is the futures price at time i with j down moves
    F = np.full((opt_periods + 1, opt_periods + 1), np.nan)
    for i in range(opt_periods + 1):
        # remaining time from option node i to futures expiry
        carry = np.exp((r - c) * (fut_periods - i) * dt)
        F[i, :i+1] = S[i, :i+1] * carry

    # Terminal payoff at option maturity
    V = np.maximum(F[opt_periods, :opt_periods + 1] - K, 0.0)

    # Backward induction with early exercise
    for i in range(opt_periods - 1, -1, -1):

        # Continuation value under the risk-neutral measure
        cont = disc * (p * V[:i+1] + (1 - p) * V[1:i+2])

        # Immediate exercise value at time i
        exer = np.maximum(F[i, :i+1] - K, 0.0)

        # American option value
        V = np.maximum(cont, exer)

        # Diagnostic output to identify early exercise
        print("\nTime step i =", i)
        print("Immediate exercise value:", exer)
        print("Continuation value:", cont)

    return V

In [20]:
FV = american_call_on_futures_from_stock_tree(S, K, u, d, r, c, T, N, opt_periods=10, fut_periods=15)


Time step i = 9
Immediate exercise value: [31.84553547 21.27298635 11.48846905  2.43324709  0.          0.
  0.          0.          0.          0.        ]
Continuation value: [31.83492206 21.26589653 11.48464019  3.37412447  0.          0.
  0.          0.          0.          0.        ]

Time step i = 8
Immediate exercise value: [26.47965015 16.30705078  6.89267271  0.          0.          0.
  0.          0.          0.        ]
Continuation value: [26.47082507 16.301616    7.36773983  1.66110145  0.          0.
  0.          0.          0.        ]

Time step i = 7
Immediate exercise value: [21.3167513  11.52897195  2.47073109  0.          0.          0.
  0.          0.        ]
Continuation value: [21.3096469  11.76616004  4.46995971  0.81777008  0.          0.
  0.          0.        ]

Time step i = 6
Immediate exercise value: [16.34916015  6.93164343  0.          0.          0.          0.
  0.        ]
Continuation value: [16.46405127  8.06043259  2.61549212  0.40259305  0

In [21]:
round(FV[0],2)

np.float64(1.66)